In [1]:
import mido
from mido import MidiFile, MidiTrack, Message
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
import numpy as np


C:\Users\kpgaj\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\kpgaj\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\kpgaj\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])

In [2]:
#mid = MidiFile(r"E:/music/Durga.mid") 
mid = MidiFile(r"D:/muic.mid") 
notes = []

In [3]:
print(mid)

<midi file 'D:/muic.mid' type 0, 1 tracks, 33546 messages>


In [7]:
notes = []
for msg in mid:   
    notes.append(msg.bytes()[1])
        


In [8]:

scaler = MinMaxScaler()
scaler.fit(np.array(notes).reshape(-1,1))
notes = list(scaler.transform(np.array(notes).reshape(-1,1)))

C:\Users\kpgaj\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int32 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [9]:

notes = [list(note) for note in notes]


X = []
y = []

n_prev = 55
for i in range(len(notes)-n_prev):
    X.append(notes[i:i+n_prev])
    y.append(notes[i+n_prev])

    
X_test = X[-300:]
X = X[:-300]
y = y[:-300]

In [10]:
from keras.callbacks import ModelCheckpoint, History
model = Sequential()
model.add(LSTM(256, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(128, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(64, input_shape=(n_prev, 1), return_sequences=False))
model.add(Dropout(0.6))
model.add(Dense(1))
model.add(Activation('linear'))
optimizer = Adam(lr=0.001)
model.compile(loss='mse', optimizer=optimizer)
history = History()


In [12]:

model.fit(np.array(X), np.array(y),642, 5, verbose=1, callbacks=[history])

Epoch 1/5
33191/33191 [==============================] - 183s 6ms/step - loss: 0.0502
Epoch 2/5
33191/33191 [==============================] - 209s 6ms/step - loss: 0.0309
Epoch 3/5
33191/33191 [==============================] - 157s 5ms/step - loss: 0.0295
Epoch 4/5
33191/33191 [==============================] - 133s 4ms/step - loss: 0.0287
Epoch 5/5
33191/33191 [==============================] - 134s 4ms/step - loss: 0.0286


In [13]:
prediction = model.predict(np.array(X_test))
prediction = np.squeeze(prediction)
prediction = np.squeeze(scaler.inverse_transform(prediction.reshape(-1,1)))
prediction = [int(i) for i in prediction]

In [15]:
mid = MidiFile()
track = MidiTrack()
t = 0
for note in prediction:
    # 147 means note_on
    # 67 is velosity
    note = np.asarray([147, note, 67])
    bytes = note.astype(int)
    msg = Message.from_bytes(bytes[0:3])
    t += 1
    msg.time = t
    track.append(msg)
mid.tracks.append(track)
mid.save('LSTM_muic.mid')